In [5]:
import pandas as pd
import nltk.translate.bleu_score as bleu
from datasets import load_dataset

/Users/anshulwadhawan/opt/anaconda3/envs/cenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = load_dataset("google/fleurs", 'af_za')

Extracting data files: 100%|██████████| 3/3 [00:09<00:00,  3.02s/it]


Dataset fleurs downloaded and prepared to /Users/anshulwadhawan/.cache/huggingface/datasets/google___fleurs/af_za/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 41.10it/s]


In [9]:
df = pd.DataFrame(dataset['train'])

In [11]:
df.head()

,id,num_samples,path,audio,transcription,raw_transcription,gender,lang_id,language,lang_group_id
0,766,72960,/Users/anshulwadhawan/.cache/huggingface/datas...,"{'path': 'train/10010386886416577453.wav', 'ar...",families met kinders is nie 'n algemene versky...,Families met kinders is nie ‘n algemene versky...,1,0,Afrikaans,3
1,22,52320,/Users/anshulwadhawan/.cache/huggingface/datas...,"{'path': 'train/10017360593491868518.wav', 'ar...",hsieh het ook geredeneer dat die fotogeniese m...,Hsieh het ook geredeneer dat die fotogeniese M...,1,0,Afrikaans,3
2,361,182400,/Users/anshulwadhawan/.cache/huggingface/datas...,"{'path': 'train/10020081195348097870.wav', 'ar...",die volgende prent toon die atome wat protone ...,Die volgende prent toon die atome wat protone ...,1,0,Afrikaans,3
3,1340,242880,/Users/anshulwadhawan/.cache/huggingface/datas...,"{'path': 'train/10020481472273143178.wav', 'ar...",dit was op hierdie tyd dat die oordrag van die...,Dit was op hierdie tyd dat die oordrag van die...,0,0,Afrikaans,3
4,1469,66240,/Users/anshulwadhawan/.cache/huggingface/datas...,"{'path': 'train/10050183618291444534.wav', 'ar...",gosling en stone het nominasies vir onderskeid...,"Gosling en Stone het nominasies vir, onderskei...",1,0,Afrikaans,3


In [10]:
# set up the BLEU score calculation
weights = (0.25, 0.25, 0.25, 0.25)  # uniform weights for 1-gram to 4-gram
smooth_func = bleu.SmoothingFunction().method1  # use smoothing method 1


In [12]:
# define a function to calculate BLEU scores for each row
def calculate_bleu(row):
    # get the relevant columns
    ref = row['raw_transcription']
    candidate = row['transcription']
    # calculate the BLEU score
    bleu_score = bleu.sentence_bleu([ref.split()], candidate.split(), weights, smoothing_function=smooth_func)
    return bleu_score


In [13]:
# apply the function to the dataframe and create a new column for the BLEU scores
df['bleu_score'] = df.apply(calculate_bleu, axis=1)

In [19]:
df[['bleu_score', 'raw_transcription', 'transcription']].head()

,bleu_score,raw_transcription,transcription
0,0.563258,Families met kinders is nie ‘n algemene versky...,families met kinders is nie 'n algemene versky...
1,0.578357,Hsieh het ook geredeneer dat die fotogeniese M...,hsieh het ook geredeneer dat die fotogeniese m...
2,0.724551,Die volgende prent toon die atome wat protone ...,die volgende prent toon die atome wat protone ...
3,0.573742,Dit was op hierdie tyd dat die oordrag van die...,dit was op hierdie tyd dat die oordrag van die...
4,0.039074,"Gosling en Stone het nominasies vir, onderskei...",gosling en stone het nominasies vir onderskeid...
